In [214]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
import re             # regular expression library
from wordcloud import WordCloud
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
from collections import Counter
from gensim.models.phrases import Phrases, Phraser
#from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [215]:
pwd

'/Users/Robert/DSI/Capstone_3'

In [216]:
# Read datasets/papers.csv into papers
papers = pd.read_csv('nips-papers/papers.csv')


In [217]:
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [218]:
papers.tail()

,id,year,title,event_type,pdf_name,abstract,paper_text
7236,994,1994,Single Transistor Learning Synapses,NaN,994-single-transistor-learning-synapses.pdf,Abstract Missing,Single Transistor Learning Synapses\n\nPaul Ha...
7237,996,1994,"Bias, Variance and the Combination of Least Sq...",NaN,996-bias-variance-and-the-combination-of-least...,Abstract Missing,"Bias, Variance and the Combination of\nLeast S..."
7238,997,1994,A Real Time Clustering CMOS Neural Engine,NaN,997-a-real-time-clustering-cmos-neural-engine.pdf,Abstract Missing,A Real Time Clustering CMOS\nNeural Engine\nT....
7239,998,1994,Learning direction in global motion: two class...,NaN,998-learning-direction-in-global-motion-two-cl...,Abstract Missing,Learning direction in global motion: two\nclas...
7240,999,1994,Correlation and Interpolation Networks for Rea...,NaN,999-correlation-and-interpolation-networks-for...,Abstract Missing,Correlation and Interpolation Networks for\nRe...


In [219]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 7 columns):
id            7241 non-null int64
year          7241 non-null int64
title         7241 non-null object
event_type    2422 non-null object
pdf_name      7241 non-null object
abstract      7241 non-null object
paper_text    7241 non-null object
dtypes: int64(2), object(5)
memory usage: 396.1+ KB


In [220]:
drop_cols = ['id', 'event_type', 'pdf_name']
papers.drop(columns=drop_cols, axis=1, inplace=True)


In [221]:
# Make dataframe for each half decade 1987 - 2017
papers_thru_1990 = papers[papers.year <= 1990]
papers_thru_1990.reset_index(inplace=True)

papers_thru_1995 = papers[(papers.year > 1990) & (papers.year <= 1995)]
papers_thru_1995.reset_index(inplace=True)

papers_thru_2000 = papers[(papers.year > 1995) & (papers.year <= 2000)]
papers_thru_2000.reset_index(inplace=True)

papers_thru_2005 = papers[(papers.year > 2000) & (papers.year <= 2005)]
papers_thru_2005.reset_index(inplace=True)

papers_thru_2010 = papers[(papers.year > 2005) & (papers.year <= 2010)]
papers_thru_2010.reset_index(inplace=True)

papers_thru_2015 = papers[(papers.year > 2010) & (papers.year <= 2015)]
papers_thru_2015.reset_index(inplace=True)

papers_thru_2017 = papers[(papers.year > 2015) & (papers.year <= 2017)]
papers_thru_2017.reset_index(inplace=True)


# Preprocess text

In [222]:
# Use stopwords from "nltk.corpus"
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) # sets are efficient
stop_words.update(["this", "that", "thus", "from", "does", "example", "however", "since", "given", 
                    "et", "al", "hence", "therefore", "use", "used", "note", "corresponding", 
                    "setting", "well", "one", "form", "using", "second", "even", "known", "either",
                    "consider", "particular", "general", "represent", "case", "output", "defined",
                    "rather", "though", "although", "set", "problem", "function", "figure", "given",
                    "results", "number", "time", "data", "results", "given", "information", "different",
                    "models", "method", "n", "j", "r", "l", "mj", "a", "cj", "zero", "iv", "is", "elsewhere",
                    "large", "may", "finally", "c", "describing", "l", "cells", "single", "field",
                    "approximation", "activity", "mean", "input"])


In [223]:
start_time = timeit.default_timer()
# This function removes process the stopwords. Takes 3-4 mins
def rem_stopwords(txt):
    words = txt.split()
    ret_word = words.copy()
    for w in words: 
        if w.lower() in stop_words:
            ret_word.remove(w)
    return (" ".join(ret_word))
papers_thru_2010['text_processed'] = papers_thru_2010['paper_text'].map(rem_stopwords)
elapsed = timeit.default_timer() - start_time
print(elapsed)

50.422314343000835


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [224]:
start_time = timeit.default_timer()
# This cell takes a long time, longer than 3 mins(seems slower)
#Print the text of the first 3 rows 
print(papers_thru_2010['paper_text'].head(3))

# Remove punctuation
papers_thru_2010['text_processed'] = papers_thru_2010['text_processed'].map(lambda text: [w for w in word_tokenize(text.lower()) 
                  if w.isalpha()])

# # Print the processed titles of the first rows 
papers_thru_2010['text_processed'][0:3]

elapsed = timeit.default_timer() - start_time
print(elapsed)

0    Learning Dense 3D Correspondence\n\n?\n\nFlori...
1    Effects of Stress and Genotype on Meta-paramet...
2    PAC-Bayes Bounds for the Risk of the Majority ...
Name: paper_text, dtype: object
42.25543178699991


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [225]:
#Runs fast, less than 2 secs.
# start_time = timeit.default_timer()
papers_thru_2010['text_string'] = [" ".join(map(str, l)) for l in papers_thru_2010['text_processed']]
elapsed = timeit.default_timer() - start_time
# print(elapsed)
# papers.head(2)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [226]:
print(papers_thru_2010.info())
papers_thru_2010.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
index             1225 non-null int64
year              1225 non-null int64
title             1225 non-null object
abstract          1225 non-null object
paper_text        1225 non-null object
text_processed    1225 non-null object
text_string       1225 non-null object
dtypes: int64(2), object(5)
memory usage: 67.1+ KB
None


,index,year,title,abstract,paper_text,text_processed,text_string
0,2155,2006,Learning Dense 3D Correspondence,Abstract Missing,Learning Dense 3D Correspondence\n\n?\n\nFlori...,"[learning, dense, correspondence, florian, ste...",learning dense correspondence florian steinke ...
1,2156,2006,Effects of Stress and Genotype on Meta-paramet...,Abstract Missing,Effects of Stress and Genotype on Meta-paramet...,"[effects, stress, genotype, dynamics, reinforc...",effects stress genotype dynamics reinforcement...
2,2157,2006,PAC-Bayes Bounds for the Risk of the Majority ...,Abstract Missing,PAC-Bayes Bounds for the Risk of the Majority ...,"[bounds, risk, majority, vote, variance, gibbs...",bounds risk majority vote variance gibbs class...


In [227]:
print(papers.info())
papers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 4 columns):
year          7241 non-null int64
title         7241 non-null object
abstract      7241 non-null object
paper_text    7241 non-null object
dtypes: int64(1), object(3)
memory usage: 226.4+ KB
None


,year,title,abstract,paper_text
0,1987,Self-Organization of Associative Database and ...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,1987,A Mean Field Theory of Layer IV of Visual Cort...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,1988,Storing Covariance by the Associative Long-Ter...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...


In [228]:
#print(list(papers_thru_2010.text_processed[0]))
type(papers_thru_2010.text_processed)

pandas.core.series.Series

In [229]:
# # Use stopwords from "nltk.corpus"
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english')) # sets are efficient
STOPWORDS.update(["this", "that", "thus", "from", "does", "example", "however", "since", "given", 
                    "et", "al", "hence", "therefore", "use", "used", "note", "corresponding", 
                    "setting", "well", "one", "form", "using", "second", "even", "known", "either",
                    "consider", "particular", "general", "represent", "case", "output", "defined",
                    "rather", "though", "although", "set", "problem", "function", "figure", "given",
                    "results", "number", "time", "data", "results", "given", "information", "different",
                    "models", "method", "n", "j", "r", "l", "mj", "a", "cj", "zero", "iv", "is", "elsewhere",
                    "large", "may", "finally", "c", "describing", "l", "cells", "single", "field",
                    "approximation", "activity", "mean", "input", "x", "f", "here", "y", "means", "make", "say",
                    "that", "then", "good", "clarify", "b", "t", "layers", "s", "e", "units", "unit", "input", "h", 
                    "z", "p", "xt", "items", "player", "two", "item", "let", "d", "w", "ti", "m", "g", "local",
                    "q", "actions", "users", "system", "regret", "user", "xi", "rule",
                    "eq", "kl", "em", "v", "u", "first", "entries", "optimal", "error", "o", "result",
                    "see", "section", "update", "theorem", "variables", "rl", "st", "size", "based", "fig", "source",
                    "sources", "step", "o", "xi", "vi", "approach", "yt", "c", "new", "unit", "systems", "points", "point",
                    "test", "values", "value", "fi", "examples", "representation", "processing", "shown", "inputs", "initial",
                    "many", "problems", "speaker", "yi", "effect", "gp", "ya", "ylx", "paper"])


In [230]:
print(len(STOPWORDS))
STOPWORDS

339


{'a',
 'about',
 'above',
 'actions',
 'activity',
 'after',
 'again',
 'against',
 'al',
 'all',
 'also',
 'although',
 'am',
 'an',
 'and',
 'any',
 'approach',
 'approximation',
 'are',
 "aren't",
 'as',
 'at',
 'b',
 'based',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'c',
 'can',
 "can't",
 'cannot',
 'case',
 'cells',
 'cj',
 'clarify',
 'com',
 'consider',
 'corresponding',
 'could',
 "couldn't",
 'd',
 'data',
 'defined',
 'describing',
 'did',
 "didn't",
 'different',
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'e',
 'each',
 'effect',
 'either',
 'else',
 'elsewhere',
 'em',
 'entries',
 'eq',
 'error',
 'et',
 'even',
 'ever',
 'example',
 'examples',
 'f',
 'few',
 'fi',
 'field',
 'fig',
 'figure',
 'finally',
 'first',
 'for',
 'form',
 'from',
 'function',
 'further',
 'g',
 'general',
 'get',
 'given',
 'good',
 'gp',
 'h',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',


In [231]:
def tokenize(text):
    return [token for token in text if token not in STOPWORDS]
paps = papers_thru_2010.text_processed
documents = paps.tolist()
texts = [tokenize(document) for document in documents]

In [232]:
print(type(texts))
len(texts)

<class 'list'>


1225

In [233]:
trg_sent = [['machine','learning','neural','network','dummy','neural','networks', "deep", "learning", "dum", "expert", "system", "expert","systems" ], 
            ['neural','networks', 'dummy','neural','network','machine','learning',"deep", "learning", "expert", "system", "expert","systems"]]
phrases = Phrases(trg_sent, min_count=1, threshold=1)
bigram = Phraser(phrases)
test_sent = ['machine','learning','neural','network','neural','networks', "deep", "learning", "expert", "system", "expert", "system"]
print(bigram[test_sent])

corpus = [bigram[paper] for paper in texts]

['machine_learning', 'neural_network', 'neural_networks', 'deep_learning', 'expert_system', 'expert_system']


In [234]:
print(texts[5])
corpus[111]

['registration', 'coherent', 'drift', 'andriy', 'myronenko', 'xubo', 'song', 'miguel', 'department', 'computer', 'science', 'electrical', 'engineering', 'ogi', 'school', 'science', 'engineering', 'oregon', 'health', 'science', 'university', 'beaverton', 'usa', 'myron', 'xubosong', 'miguel', 'abstract', 'introduce', 'coherent', 'drift', 'cpd', 'novel', 'probabilistic', 'nonrigid', 'registration', 'sets', 'registration', 'treated', 'maximum', 'likelihood', 'ml', 'estimation', 'motion', 'coherence', 'constraint', 'velocity', 'moves', 'coherently', 'align', 'formulate', 'motion', 'coherence', 'constraint', 'derive', 'solution', 'regularized', 'ml', 'estimation', 'variational', 'leads', 'elegant', 'kernel', 'derive', 'algorithm', 'penalized', 'ml', 'optimization', 'deterministic', 'annealing', 'cpd', 'simultaneously', 'finds', 'transformation', 'correspondence', 'sets', 'without', 'making', 'prior', 'assumption', 'transformation', 'model', 'except', 'motion', 'coherence', 'estimate', 'compl

['learnability',
 'doubling',
 'dimension',
 'li',
 'genome',
 'institute',
 'singapore',
 'philip',
 'long',
 'google',
 'plong',
 'abstract',
 'classifiers',
 'probability',
 'distribution',
 'domain',
 'define',
 'metric',
 'taking',
 'distance',
 'pair',
 'classifiers',
 'probability',
 'classify',
 'random',
 'differently',
 'prove',
 'bounds',
 'sample',
 'complexity',
 'pac',
 'learning',
 'terms',
 'doubling',
 'dimension',
 'metric',
 'bounds',
 'imply',
 'bounds',
 'sample',
 'complexity',
 'learning',
 'halfspaces',
 'respect',
 'uniform',
 'distribution',
 'constant',
 'factor',
 'prove',
 'bound',
 'holds',
 'algorithm',
 'outputs',
 'classifier',
 'whenever',
 'possible',
 'bound',
 'terms',
 'maximum',
 'doubling',
 'dimension',
 'strengthens',
 'best',
 'bound',
 'terms',
 'alone',
 'show',
 'bound',
 'doubling',
 'dimension',
 'terms',
 'contrast',
 'metric',
 'dimension',
 'introduction',
 'classifiers',
 'probability',
 'distribution',
 'domain',
 'induce',
 'metric'

In [235]:
corpus[2]

['bounds',
 'risk',
 'majority',
 'vote',
 'variance',
 'gibbs',
 'classifier',
 'alexandre',
 'lacasse',
 'franc',
 'ois',
 'laviolette',
 'mario',
 'marchand',
 'epartement',
 'universit',
 'laval',
 'qu',
 'ebec',
 'canada',
 'pascal',
 'germain',
 'epartement',
 'universit',
 'laval',
 'qu',
 'ebec',
 'canada',
 'nicolas',
 'usunier',
 'laboratoire',
 'informatique',
 'de',
 'paris',
 'universit',
 'pierre',
 'marie',
 'curie',
 'paris',
 'france',
 'abstract',
 'propose',
 'bounds',
 'risk',
 'weighted',
 'majority',
 'vote',
 'depend',
 'variance',
 'associated',
 'gibbs',
 'classifier',
 'show',
 'bounds',
 'smaller',
 'risk',
 'gibbs',
 'classifier',
 'arbitrarily',
 'close',
 'risk',
 'gibbs',
 'classifier',
 'close',
 'moreover',
 'show',
 'bounds',
 'uniformly',
 'estimated',
 'training',
 'possible',
 'posteriors',
 'moreover',
 'improved',
 'sample',
 'unlabelled',
 'introduction',
 'initiated',
 'aims',
 'providing',
 'pac',
 'guarantees',
 'learning',
 'algorithms',
 'wi

In [236]:
len(corpus)

1225

In [237]:
from collections import defaultdict
frequency = defaultdict(int)
for text in corpus:
    for token in text:
        frequency[token] += 1

texts_2 = [[token for token in text if frequency[token] > 10] for text in corpus]

In [239]:
papers_thru_2010.head(3)

,index,year,title,abstract,paper_text,text_processed,text_string
0,2155,2006,Learning Dense 3D Correspondence,Abstract Missing,Learning Dense 3D Correspondence\n\n?\n\nFlori...,"[learning, dense, correspondence, florian, ste...",learning dense correspondence florian steinke ...
1,2156,2006,Effects of Stress and Genotype on Meta-paramet...,Abstract Missing,Effects of Stress and Genotype on Meta-paramet...,"[effects, stress, genotype, dynamics, reinforc...",effects stress genotype dynamics reinforcement...
2,2157,2006,PAC-Bayes Bounds for the Risk of the Majority ...,Abstract Missing,PAC-Bayes Bounds for the Risk of the Majority ...,"[bounds, risk, majority, vote, variance, gibbs...",bounds risk majority vote variance gibbs class...


In [240]:
len(texts_2)

1225

In [241]:
from gensim import corpora
dictionary = corpora.Dictionary(texts_2)
corpus_test = [dictionary.doc2bow(text) for text in texts_2]

In [242]:
len(corpus_test)

1225

In [243]:
len(dictionary)

12879

In [244]:
start_time = timeit.default_timer()
from gensim.models.ldamodel import LdaModel
# fit gensim's LDA model
# the number of topics should be optimized iteratively
papers_topics = LdaModel(corpus=corpus_test,
                           id2word=dictionary,
                           num_topics=20,
                           passes=10)


elapsed = timeit.default_timer() - start_time
print(elapsed)

71.04320857700077


In [245]:
from pprint import pprint
# print out first 10 topics
for i, topic in enumerate(papers_topics.print_topics(20)):
    pprint("{} --- {}".format(i, topic))
    print()


('0 --- (0, \'0.021*"image" + 0.014*"images" + 0.011*"features" + '
 '0.011*"model" + 0.010*"object" + 0.010*"learning" + 0.009*"training" + '
 '0.007*"feature" + 0.006*"recognition" + 0.005*"labels"\')')

('1 --- (1, \'0.013*"matrix" + 0.012*"learning" + 0.010*"kernel" + '
 '0.008*"distance" + 0.008*"algorithm" + 0.006*"metric" + 0.005*"methods" + '
 '0.005*"linear" + 0.005*"optimization" + 0.005*"space"\')')

('2 --- (2, \'0.013*"state" + 0.013*"model" + 0.007*"process" + 0.005*"policy" '
 '+ 0.005*"distribution" + 0.005*"dynamics" + 0.005*"states" + 0.005*"control" '
 '+ 0.004*"parameters" + 0.004*"observations"\')')

('3 --- (3, \'0.016*"algorithm" + 0.013*"learning" + 0.010*"policy" + '
 '0.007*"image" + 0.006*"state" + 0.005*"algorithms" + 0.005*"model" + '
 '0.005*"action" + 0.005*"linear" + 0.004*"online"\')')

('4 --- (4, \'0.014*"bound" + 0.012*"loss" + 0.010*"learning" + '
 '0.008*"algorithm" + 0.007*"bounds" + 0.007*"distribution" + '
 '0.007*"probability" + 0.006*"function

In [246]:
# Takes about 5-10 minutes, depends how many papers
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(papers_topics, corpus_test, dictionary)
pyLDAvis.display(vis_data)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
